In [ ]:
import sys
import os
import pandas as pd 
import numpy as np

sys.path.append('./..')
from py_helpers.gpt import get_prompts 
from dotenv import load_dotenv
from py_helpers.sqlite import SQLiteConn
from datetime import datetime
import json 

sqlite = SQLiteConn('gpt_generated_v2.db')
load_dotenv('./.env')

## Testing

In [ ]:
system_prompt_v1 =\
""" 
Generate 100 one-sentence summaries of conversations between a user and an AI assistant. It should be a mix of standard user-assistant instructional questions, as well as some unusual conversations, including some that are creative, unusual, as well as others that are deeply scientific.
- IMPORTANT: BE CREATIVE and think of unusual conversations!
- Each summary should be one sentence. 
- Do NOT return similar conversation topics!
- Avoid conversations that are extremely generic, like "user asks for tips on time management"
- Return your results as a JSON array. 

Here is an example of 5 good topics:
[
    "User wants to learn about the space and the potential of extraterrestrial life, especially relating to the Dark Forest hypothesis",
    "User wants to rant about his day and express his frustrations about his relationship with his children, who are always on their cellphones",
    "User wants to talk to the assistant and solicit the assistant's feelings about the decline of birth rates globally",
    "User wants the assistant to talk like an animal while writing poetry",
    "User wants to make back-and-forth dad jokes with the assistant"
]
"""

In [ ]:
topics_to_avoid = ['User wants to grow a vegetable garden and asks the assistant for help']

prompts_list = [{'role': 'system', 'content': system_prompt_v1 + '\n' + 'Do NOT generate topics very similar to these existing topics: ' + json.dumps(topics_to_avoid)}]

res = await get_prompts(
    [prompts_list],
    {'model': 'gpt-4o', 'temperature': 1.0, 'response_format': {'type': 'json_object'}}, 
    api_key = os.environ.get('OPENAI_API_KEY')
)
res

In [ ]:
sqlite.execute(
    """
    CREATE TABLE IF NOT EXISTS topics (
        id INTEGER PRIMARY KEY,
        prompt STRING NOT NULL,
        topic STRING NOT NULL,
        added_at STRING NOT NULL 
    )
    """
)

display(sqlite.get_query('SELECT * FROM topics ORDER BY added_at DESC'))

In [ ]:
def parse_response(r, version):
    try:
        parsed = json.loads(r['choices'][0]['message']['content'])
        pairs = parsed['conversation_pairs']
        cleaned = []
        for pair in pairs:
            try:
                topic = pair['topic']
                generic = pair['generic']
                dog = pair['dog_related']
                cleaned.append({
                    'prompt': version,
                    'topic': topic,
                    'generic': json.dumps(generic),
                    'dog': json.dumps(dog),
                    'added_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
            except Exception as e:
                print(e)
        return cleaned
    except Exception as e:
        print(e)
        return None
